In [ ]:
a = {'a': {3, 4}}
b = {'a': [1, 2]}
a['a'].update(b['a'])
a

In [ ]:
import shelve

class fdict(dict):
    '''Flattened nested dict, all items are settable and gettable through ['item1']['item2'] standard form or ['item1/item2'] internal form.
    This allows to replace the internal dict with any on-disk storage system like a shelve's shelf (great for huge nested dicts that cannot fit into memory).
    Main limitation: an entry can be both a singleton and a nested fdict, and there is no way to tell what is what, no error will be shown, the singleton will always be returned.
    '''
    def __init__(self, d=None, rootpath='', delimiter='/', *args):
        if d:
            self.d = d
        else:
            self.d = {}
        self.rootpath = rootpath
        self.delimiter = delimiter
        #return dict.__init__(self, *args)

    def _buildpath(self, key):
        return self.rootpath+self.delimiter+key if self.rootpath else key

    def __getitem__(self, key):
        # Node or leaf?
        if key in self.d: # Leaf: return the value
            return self.d.__getitem__(key)
        else: # Node: return a new full fdict based on the old one but with a different rootpath to limit the results by default
            return fdict(d=self.d, rootpath=self._buildpath(key))
        #return dict.__getitem__(self, key)

    def __setitem__(self, key, value):
        #fullkey = self._buildpath(key)
        #if fullkey in self.d and :
        #    raise ValueError('Conflict detected: the following key is both a singleton and a nested dict: %s' % fullkey)
        self.d.__setitem__(self._buildpath(key), value)
        #dict.__setitem__(self, key, value)

    #def addchild(self, key, value=None):
    #    self.d[self._buildpath(key)] = value

    def keys(self):
        if not self.rootpath:
            return self.d.keys()
        else:
            pattern = self.rootpath+self.delimiter
            lpattern = len(pattern)
            return [k[lpattern:] for k in self.d.keys() if k.startswith(pattern)]

    def items(self):
        # Filter items to keep only the ones below the rootpath level
        if not self.rootpath:
            return self.d.items()
        else:
            pattern = self.rootpath+self.delimiter
            lpattern = len(pattern)
            return [(k[lpattern:], v) for k,v in self.d.items() if k.startswith(pattern)]

    def values(self):
        if not self.rootpath:
            return self.d.values()
        else:
            pattern = self.rootpath+self.delimiter
            lpattern = len(pattern)
            return [v for k,v in self.d.items() if k.startswith(pattern)]

    def update(self, d2):
        return self.d.update(d2.d)
    
    def __repr__(self):
        # Filter the items if there is a rootpath and return as a new fdict
        if self.rootpath:
            return repr(fdict(d=dict(self.items())))
        else:
            return self.d.__repr__()

    def __str__(self):
        if self.rootpath:
            return str(fdict(d=dict(self.items())))
        else:
            return self.d.__str__()

class sfdict(fdict):
    '''A nested dict with flattened internal representation, combined with shelve to allow for efficient storage and memory allocation of huge nested dictionnaries.
    If you change leaf items (eg, list.append), do not forget to sync() to commit changes to disk and empty memory cache because else this class has no way to know if leaf items were changed!
    '''
    def __init__(self, *args, **kwargs):
        if not ('filename' in kwargs):
            self.filename = None
        else:
            self.filename = kwargs['filename']
            del kwargs['filename']
        fdict.__init__(self, *args, **kwargs)
        self.d = shelve.open(filename=self.filename, flag='c', writeback=True)

    def __setitem__(self, key, value):
        fdict.__setitem__(self, key, value)
        self.sync()

    def get_filename(self):
        return self.filename

    def sync(self):
        self.d.sync()

    def close(self):
        self.d.close()
    

a = fdict()
a['a'] = {}
a['c']['b'] = set([1, 2])
b = a['a']
print(a.keys())
print(b.items())
print(a.items())
c = {}
c['b'] = set([1, 2])
print(c.items())

print(a.values())
print(b.values())
print(c.values())

d = fdict()
d['b'] = {'a': 1}
d['c/b'] = set([2, 3, 5])
print(d)

a.update(d)
print(a)
print(a['c'])

In [ ]:
a['a/b'] = 3
a['c']
a['c']['e'] = 1
a

In [ ]:
g = sfdict(filename='testshelf')
g['a'] = 3
g['b/c'] = set([1, 3, 4])
g['d'] = {}
g

In [ ]:
h = sfdict(filename='testshelf')
h